In [ ]:
#Valentine Shilov 
#LongTailRelevance final ranker
import gzip
import bz2
import operator
from collections import defaultdict
from multiprocessing import Pool
import os
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
MAX_SHOWS=20
class doc:
    def __init__(self):
        global MAX_SHOWS
        self.nclicks = 0
        self.nshows = 0
        self.pos_clicks = np.zeros(MAX_SHOWS, dtype=np.int)
        self.view_times = []
        self.nlast = 0
        self.nbefore=0
        self.nafter=0
        self.pos_done_clicks = np.zeros(MAX_SHOWS, dtype=np.int)
        self.url = ""
        self.avg_time_cnt=0
        self.avg_time = 0
        self.ld=100000000
        self.ldc=100000000
        self.pos_shows = np.zeros(MAX_SHOWS, dtype=np.int)
    def update_lev(self, query):
        global docToQuery
        global urls
        if self.url in urls:
            did = int(urls[self.url])
            doc_queries = docToQuery[did]
            for q in doc_queries:
                ld = distance(query, q)
                self.ld = min(self.ld, ld)
    def update_lev_clicked(self, query):
        global docToQuery
        global urls
        if self.url in urls:
            
            did = int(urls[self.url])
            
            doc_queries = docToQuery[did]
            #print(doc_queries)
            for q in doc_queries:
                #print("sfafdafsf")
                ld = distance(query, q)
                #print(ld)
                self.ldc = min(self.ldc, ld)
    def __str__(self):
        a = np.nonzero(self.pos_clicks)[0]
        if(a.shape[0]==0):
            a = np.zeros(1, dtype=np.int)
        b = self.pos_clicks[0:np.max(a)+1]
        if len(self.view_times)>0:
            self.avg_time = np.mean(np.array(self.view_times))
        else: 
            #avg_time = 0
            pass
        avg_time = self.avg_time
        aa = np.nonzero(self.pos_done_clicks)[0]
        if(aa.shape[0]==0): 
            aa = np.zeros(1, dtype=np.int)
        bb = self.pos_done_clicks[0:np.max(aa)+1]

        aaa = np.nonzero(self.pos_shows)[0]
        if(aaa.shape[0]==0): 
            aaa = np.zeros(1, dtype=np.int)
        bbb = self.pos_shows[0:np.max(aaa)+1]
        
        rv = self.url + "\t" + str(self.nclicks) + "\t" + str(self.nshows) + "\t" + str(self.nlast) + "\t" + str(avg_time) + "\t"+ str(b).replace("[", "").replace("]", "").replace("\t", " ").strip() + "\t" + str(bb).replace("[", "").replace("]", "").replace("\t", " ").strip()  + "\t" + str(self.ld)  + "\t" + str(self.ldc)+ "\t" + str(self.nbefore) + "\t" + str(self.nafter) + "\t" + str(bbb).replace("[", "").replace("]", "").replace("\t", " ").strip()
        return rv.replace("\n", "")


    def __repr__(self):
        return self.__str__()
    def agr_from_str(self, line):
        parts = line.split("\t")
        if len(parts)<=10:
            #print("error", len(parts), parts)
            #return
            pass
        #print(len(parts), line)
        self.nclicks += int(parts[1])
        self.nshows += int(parts[2])
        self.nlast += int(parts[3])
        self.url = parts[0]
        at = float(parts[4])
        if self.avg_time_cnt==0 and at!=0:
            self.avg_time = at
            self.avg_time_cnt+=1
        elif at!=0:
            self.avg_time_cnt+=1
            self.avg_time =  (self.avg_time*(self.avg_time_cnt-1) + at) / (self.avg_time_cnt)
        pks = parts[5].split(" ")
        pks = list(filter(lambda x: len(x)>0, pks))
        #print(pks)
        pk = list(map(int, pks))
        
        pk = np.array(pk)
        pk = pk[0:MAX_SHOWS]
        self.pos_clicks[0:len(pk)] += pk

       
        ppks = parts[6].split(" ")
        ppks = list(filter(lambda x: len(x)>0, ppks))
        ppk = list(map(int, ppks))
        
        ppk = np.array(ppk)
        ppk = ppk[0:MAX_SHOWS]
        self.pos_done_clicks[0:len(ppk)] += ppk
        if len(parts)<=8:
            return
        self.ld=min(int(parts[7]), self.ld)
        self.ldc=min(int(parts[8]), self.ldc)
        if len(parts)<=10:
            #print("error", len(parts), parts)
            return
        self.nbefore+=int(parts[9])
        self.nafter+=int(parts[10])



        pppks = parts[11].split(" ")
        pppks = list(filter(lambda x: len(x)>0, pppks))
        pppk = list(map(int, pppks))
        
        pppk = np.array(pppk)
        pppk = pppk[0:MAX_SHOWS]
        self.pos_shows[0:len(pppk)] += pppk
        
        
    def features(self):
        nshows = (self.nshows - self.nclicks)
        max_pos_show=0
        max_pos_click=0
        #last_pos_click = 0
        avg_last_pos_click = 0
        if (nshows!=0) and (self.nshows!=0):
            
            ctr = self.nclicks/(nshows)
            if np.sum(self.pos_shows)>0:
                pos_clicks = ( (1.0*self.pos_clicks)/(self.pos_shows+0.000001)).tolist()
            else:
                pos_clicks = ( (1.0*self.pos_clicks)/(nshows)).tolist()
            pos_done_clicks = ( (1.0*self.pos_clicks)/nshows).tolist()
            max_pos_show = np.argmax(self.pos_shows)
            max_pos_click =  np.argmax(self.pos_clicks)
            
            if(self.nclicks!=0):
                nbefore =  (1.0*self.nbefore) / self.nclicks
                nafter =  (1.0*self.nafter) / self.nclicks
                avg_last_pos_click = np.argmax(self.pos_done_clicks)
            else:
                nbefore =  -1
                nafter =  -1
            clickEntropy = -np.sum( (1.0*self.pos_clicks)/nshows * np.log( (1.0*self.pos_clicks)/nshows + 0.0001))
        else:
            ctr = 0
            nbefore = -1
            nafter = -1
            pos_clicks = self.pos_clicks.tolist()
            pos_done_clicks = self.pos_done_clicks.tolist()
            clickEntropy=0
  
        avg_time_cnt = self.avg_time
        nlast = self.nlast
        
        #self.nafter=0
        return [ctr] + [nlast] + [avg_time_cnt] + pos_clicks + pos_done_clicks + [self.ld] + [self.ldc] + [nafter] + [nbefore] + [np.log(1.0 + self.nclicks)] + [np.log(1.0 + self.nshows - self.nclicks)] + [clickEntropy] + [max_pos_show] + [max_pos_click] + [avg_last_pos_click]
    def qfeatures(self):
        if(self.nshows!=0):
            ctr = self.nclicks/(self.nshows - self.nclicks)
            if np.sum(self.pos_shows)>0:
                pos_clicks = ( (1.0*self.pos_clicks)/(self.pos_shows+0.000001)).tolist()
            else:
                pos_clicks = ( (1.0*self.pos_clicks)/(self.nshows)).tolist()
            pos_done_clicks = (self.pos_clicks/self.nshows).tolist()
            if(self.nclicks!=0):
                nbefore =  self.nbefore / self.nclicks
                nafter =  self.nafter / self.nclicks
            else:
                nbefore =  -1
                nafter =  -1
        else:
            ctr = 0
            nbefore = 0
            nafter = 0
            pos_clicks = self.pos_clicks.tolist()
            pos_done_clicks = self.pos_done_clicks.tolist()

        avg_time_cnt = self.avg_time
        nlast = self.nlast

        self.nafter=0
        return [ctr] + [nlast] + [avg_time_cnt] + pos_clicks + [nafter] + [nbefore] + [np.log(1.0 + self.nclicks)] + [np.log(1.0 + self.nshows - self.nclicks)]
#docs = defaultdict(doc)

In [ ]:
def url_normalize(url):
    url = url.strip()
    url = url.replace("http://", "")
    if len(url)>1:
        if url[-1] == "/":
            url = url[:-1]
        url = url.lower()
        url = url.replace(":80", "")
        url = url.replace("www.", "")
        url = url.replace("//", "")
    return url
def extract_domain(url):
    global rubrics_list
    url = url.strip()
    #for r in rubrics_list:
        #url = url.replace("," + r + "," , "")    
    #    url = url.replace("," + r, "")    
    url = url.replace("http://", "")
    url = url.replace("ftp://", "")
    if len(url)>1:
        if url[-1] == "/":
            url = url[:-1]
        url = url.lower()
        
        url = url.replace("s://", "")
        url = url.replace(":80", "")
        url = url.replace("://", "")
        url = url.replace("www.", "")
        url = url.replace("//", "")
        pos = url.find("#")
        if pos>0:
            url = url[0:pos]

        pos = url.find("/")
        if pos>0:
            url = url[0:pos]
    return url

In [ ]:
def loadUrls(filename):
    urlToDid = {}
    didToUrl = {}
    domainToDid = defaultdict(list)
    didToDomain = defaultdict(list)
   
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            uid, url = line.split("\t")
            uid = int(uid)
            #urls["http://" + url] = uid
            #urls["http://" + url + "/"] = uid
            didToUrl[url_normalize(url)] = uid
            didToUrl[uid] = url_normalize(url)
            domain = extract_domain(url)
            domainToDid[domain].append(uid)
            didToDomain[uid] = domain
    return urlToDid,didToUrl,domainToDid,didToDomain
urlToDid,didToUrl,domainToDid,didToDomain = loadUrls("url.data")
urls = urlToDid

In [ ]:
DIRPREFIX = "../data/2017/"
RESPREFIX = "./"

def processFile(filename, alldocs, urlToDid, urlKey=False):
    try:
        f = open(filename,'r')
        cnt=0
        for linenum,line in enumerate(f):
            try:
                line = line[:-1]
                parts = line.split("\t", maxsplit=1)
                if not urlKey:
                    uid = parts[0]
                    uid = int(uid)
                else:
                    parts2 = line.split("\t")
                    url = parts2[1]
                    url = url.strip()
                    uid = url
                alldocs[uid].agr_from_str(parts[1])
            except Exception as e:
                print("Error ", filename,  repr(e), linenum)    
        print("Processed "+ filename +" " + str(cnt))
        cnt+=1
        

    except Exception as e:
        print("Error ", filename,  repr(e))  
        

In [ ]:
foundQueries = defaultdict(doc)
closeQueries = defaultdict(doc)
closeQueriesDomain = defaultdict(doc)
foundQueriesDomains = defaultdict(doc)
alldocs = defaultdict(doc)
alldomains = defaultdict(doc)
qid_did_features = np.load("qid_did_features.npy", allow_pickle=True).item()
content_tfidf_features = np.load("content_tfidf_data.npy", allow_pickle=True).item()
processFile("features_14.txt", alldocs,  urlToDid, urlKey=False)
processFile("features_domain_14.txt", alldomains,  urlToDid, urlKey=True)

processFile("found_queries.txt.bz2.txt", foundQueries,  urlToDid, urlKey=False)

processFile("found_queries_domain.txt.bz2.txt", foundQueriesDomains,  urlToDid, urlKey=True)



Processed features_14.txt 0
Processed features_domain_14.txt 0
Processed found_queries.txt.bz2.txt 0
Processed found_queries_domain.txt.bz2.txt 0


In [ ]:
closeQueriesDomain={}

In [ ]:
def loadCsvFeatures(filename, sep=',', header=False):
    features = {}
    with open(filename) as f:
        for line in f:
            parts = line.split(sep)
            if len(parts)>=3:
                qid = int(parts[0])
                did = int(parts[1])
                fl = []
                for feature in parts[2:]:
                    fl.append(float(feature))
                features[(qid,did)] = fl
    return features

In [ ]:
prev_ratings = loadCsvFeatures("prev_ratings.txt", sep=',', header=False)

In [ ]:
tfidf_ngram_features = loadCsvFeatures("tf_new.txt", sep=',', header=False)
tfidf_features = loadCsvFeatures("tfidf.txt", sep=',', header=False)
tf_content = loadCsvFeatures("q_doc_dist.txt", sep=',', header=False)
closeQueries = loadCsvFeatures("fasttext.txt", sep=',', header=False)

In [ ]:
bm25_features = loadCsvFeatures("bm25_use_merge_2.txt", sep=',', header=False)

In [ ]:
bm25_features[(2, 135041)]

[0.0, 0.12625268, 23.65436, 0.12625268, 0.0, 0.09042962]

In [ ]:
DOC_FEATURE_SIZE = len(alldocs[154787].features())
#DOC_FEATURE_SIZE = 0
DOMAIN_FEATURE_SIZE = len(alldomains["ask.fm"].features())
BM_FEATURES_SIZE = len(bm25_features[(2, 135041)])
FOUND_QUERIES_FEATURES_SIZE =  len(foundQueries[398601].features())
FOUND_QUERIES_DOMAINS_FEATURES_SIZE = len(foundQueriesDomains["howtolearn.ru"].features())
CLOSE_QUERIES_FEATURE_SIZE = 1#len(closeQueries[398601].features())
CLOSE_QUERIES_DOMAIN_FEATURE_SIZE = 0#len(closeQueriesDomain["howtolearn.ru"].features())
QID_DID_FEATURES_SIZE = len(qid_did_features[(3327, 64383)])
#DOMAIN_FEATURE_SIZE = 0
TFIDF_FEATURES_SIZE = len(tfidf_features[(2, 135041)])
TFIDF_NGRAM_FEATURES_SIZE = len(tfidf_ngram_features[(2, 135041)])
TF_CONTENT_FEATURES_SIZE =  len(tf_content[(2, 135041)])
#BM_FEATURES_SIZE = 0
N_S_FEATURES = len(prev_ratings[(0, 118027)])
CONTENT_TFIDF_FEATURES_SIZE = len(content_tfidf_features[(5624, 193655)])
ALL_FEATURES_SIZE = CONTENT_TFIDF_FEATURES_SIZE + QID_DID_FEATURES_SIZE+ DOC_FEATURE_SIZE + CLOSE_QUERIES_DOMAIN_FEATURE_SIZE + CLOSE_QUERIES_FEATURE_SIZE + TFIDF_FEATURES_SIZE + TF_CONTENT_FEATURES_SIZE + TFIDF_NGRAM_FEATURES_SIZE + DOMAIN_FEATURE_SIZE + BM_FEATURES_SIZE + N_S_FEATURES + FOUND_QUERIES_FEATURES_SIZE + FOUND_QUERIES_DOMAINS_FEATURES_SIZE

print(DOC_FEATURE_SIZE,QID_DID_FEATURES_SIZE, CONTENT_TFIDF_FEATURES_SIZE, CLOSE_QUERIES_DOMAIN_FEATURE_SIZE, CLOSE_QUERIES_FEATURE_SIZE, N_S_FEATURES,DOMAIN_FEATURE_SIZE, TF_CONTENT_FEATURES_SIZE, BM_FEATURES_SIZE, TFIDF_FEATURES_SIZE,TFIDF_NGRAM_FEATURES_SIZE ,FOUND_QUERIES_FEATURES_SIZE, FOUND_QUERIES_DOMAINS_FEATURES_SIZE, ALL_FEATURES_SIZE)

53 53 6 0 1 2 53 1 6 3 6 53 53 290


In [ ]:
def getFeaturesByIds(qid, did, feature_type, debug=None):
    if debug is None:
        debug=False
    global alldocs
    global alldomains
    global closeQueriesDomain
    global DOC_FEATURE_SIZE
    global DOMAIN_FEATURE_SIZE
    global ALL_FEATURES_SIZE
    global BM_FEATURES_SIZE
    global FOUND_QUERIES_FEATURES_SIZE
    global FOUND_QUERIES_DOMAINS_FEATURES_SIZE
    global TFIDF_FEATURES_SIZE
    global TFIDF_NGRAM_FEATURES_SIZE
    global TF_CONTENT_FEATURES_SIZE
    global CLOSE_QUERIES_FEATURE_SIZE
    global CLOSE_QUERIES_DOMAIN_FEATURE_SIZE
    global CONTENT_TFIDF_FEATURES_SIZE
    global closeQueries
    global content_tfidf_features
    global tf_content
    global bm25_features
    global prev_ratings
    global foundQueries
    global foundQueriesDomains
    global tfidf_features
    global tfidf_ngram_features
    global QID_DID_FEATURES_SIZE
    global qid_did_features

    default_value = 0#-1000
    flag=False

    if CONTENT_TFIDF_FEATURES_SIZE>0:
        if (qid,did) in content_tfidf_features:
            contenttfidf = content_tfidf_features[(qid,did)]
            #flag=True
            
        else:
            contenttfidf = [default_value]*CONTENT_TFIDF_FEATURES_SIZE
    else:
        contenttfidf=[]
        if debug:
            print("contenttfidf off")
    if debug:
        print("contenttfidf ", contenttfidf)



    if QID_DID_FEATURES_SIZE>0:
        if (qid,did) in qid_did_features:
            qiddidfeatures = qid_did_features[(qid,did)]
            #flag=True
            
        else:
            qiddidfeatures = [default_value]*QID_DID_FEATURES_SIZE
    else:
        qiddidfeatures=[]
        if debug:
            print("qiddidfeatures off")
    if debug:
        print("qiddidfeatures ", qiddidfeatures)


    if DOC_FEATURE_SIZE>0:
        if did in alldocs:
            docFeatures = alldocs[did].features()
            flag=True
            
        else:
            docFeatures = [default_value]*DOC_FEATURE_SIZE
    else:
        docFeatures=[]
        if debug:
            print("docFeatures off")
    if debug:
        print("docFeatures ", docFeatures)
    if DOMAIN_FEATURE_SIZE>0:
        if did in didToDomain:
            domain = didToDomain[did]
            if domain in alldomains:
                domainFeatures = alldomains[domain].features()
                flag=True
            else:
                domainFeatures = [default_value]*DOMAIN_FEATURE_SIZE
        else:
            domainFeatures = [default_value]*DOMAIN_FEATURE_SIZE
            
    else:
        domainFeatures=[]
        if debug:
            print("domain Features off")
    if debug:
        print("domainFeatures ", domainFeatures)
    if BM_FEATURES_SIZE>0:    
        if (qid, did) in bm25_features:
            bm_features = bm25_features[(qid, did) ]
        else:
            bm_features = [0.0]*BM_FEATURES_SIZE
    else:
        bm_features = []
        if debug:
            print("bmFeatures off")
    if debug:
        print("bm_features ", bm_features)
    if TF_CONTENT_FEATURES_SIZE>0:    
        if (qid, did) in tf_content:
            tf_content_features = tf_content[(qid, did) ]
        else:
            tf_content_features = [0.0]*TF_CONTENT_FEATURES_SIZE
    else:
        tf_content_features = []
        if debug:
            print("tf_content_features off")
    if debug:        
        print("tf_content_features ", tf_content_features)
    if TFIDF_FEATURES_SIZE>0:    
        if (qid, did) in tfidf_features:
            tf_features = tfidf_features[(qid, did) ]
            #flag=True
        else:
            tf_features = [0.0]*TFIDF_FEATURES_SIZE
    else:
        tf_features = []
        if debug:
            print("tf_features off")
    if debug:
        print("tf_features ", tf_features)
    if TFIDF_NGRAM_FEATURES_SIZE>0:    
        if (qid, did) in tfidf_ngram_features:
            tf_ngram_features = tfidf_ngram_features[(qid, did) ]
            #flag=True
        else:
            tf_ngram_features = [0.0]*TFIDF_NGRAM_FEATURES_SIZE
    else:
        tf_ngram_features = []
        if debug:
            print("tf_ngram_features off")
    if debug:
        print("tf_ngram_features ", tf_ngram_features)
    if N_S_FEATURES>0:
        if (qid, did) in prev_ratings:
            s_features = prev_ratings[(qid, did) ]
        else:
            s_features = [10000.0]*N_S_FEATURES
    else:
        s_features = []
        if debug:
            print("s_features off")
    if debug:
        print("s_features ", s_features)
    if FOUND_QUERIES_FEATURES_SIZE>0:
        if did in foundQueries:
            fqFeatures = foundQueries[did].features()
            flag=True
        else:
            fqFeatures = [default_value]*FOUND_QUERIES_FEATURES_SIZE
    else:
        fqFeatures = []
        if debug:
            print("fqFeatures off")
    if debug:        
        print("fqFeatures ", fqFeatures)
    if CLOSE_QUERIES_FEATURE_SIZE>0:
        if did in closeQueries:
            cqFeatures = closeQueries[did].features()
            flag=True
        else:
            cqFeatures = [default_value]*CLOSE_QUERIES_FEATURE_SIZE
    else:
        cqFeatures = []
        if debug:
            print("cqFeatures off")
    if debug:        
        print("cqFeatures ", cqFeatures)

    CLOSE_QUERIES_FEATURE_SIZE
    if FOUND_QUERIES_DOMAINS_FEATURES_SIZE>0:
        if did in didToDomain:
            domain = didToDomain[did]
            if domain in foundQueriesDomains:
                fqDomainFeatures = foundQueriesDomains[domain].features()
                #flag=True
            else:
                fqDomainFeatures = [default_value]*FOUND_QUERIES_DOMAINS_FEATURES_SIZE
        else:
            fqDomainFeatures = [default_value]*FOUND_QUERIES_DOMAINS_FEATURES_SIZE
    else:
        fqDomainFeatures=[]
        if debug:
            print("fqDomainFeatures off")
    if CLOSE_QUERIES_DOMAIN_FEATURE_SIZE>0:
        if did in didToDomain:
            domain = didToDomain[did]
            if domain in closeQueriesDomain:
                cqDomainFeatures = closeQueriesDomain[domain].features()
                #flag=True
            else:
                cqDomainFeatures = [default_value]*CLOSE_QUERIES_DOMAIN_FEATURE_SIZE
        else:
            cqDomainFeatures = [default_value]*CLOSE_QUERIES_DOMAIN_FEATURE_SIZE
    else:
        cqDomainFeatures=[]
        if debug:
            print("fqDomainFeatures off")
    if debug:   
        print("cqDomainFeatures ", cqDomainFeatures)
    res_features = (docFeatures + domainFeatures +contenttfidf+ bm_features + tf_features + tf_ngram_features + s_features + fqFeatures + fqDomainFeatures + tf_content_features + cqFeatures + cqDomainFeatures + qiddidfeatures)
    if(len(res_features)!=ALL_FEATURES_SIZE):
        print("Error!!!!! Feature size not match",len(res_features) )
        return
    return flag, res_features

In [ ]:
N_FEATURES = len(getFeaturesByIds(2,135041, feature_type=1, debug=True)[1])

In [ ]:
def makeTrain(train_marks_filename, N_FEATURES, isTest=False, feature_func=getFeaturesByIds):

    trainDidToLine = {}
    rejectedDids = defaultdict(list)
    qidToDocs = defaultdict(list)
    with open(train_marks_filename) as f:
        i=0
        nlines = 0
        for line in f:
            nlines+=1
        
        X_train = np.zeros( [nlines, N_FEATURES], dtype=float)
        y_train = np.zeros( nlines, dtype=int)
        qids = np.zeros( nlines, dtype=int)
        i=0
        f.seek(0)
        if isTest:
            f.readline()
        for ii,line in enumerate(f):
            if not isTest:
                parts = line.split("\t")
                if len(parts)!=3:
                    continue
                qid, did, mark = parts
                qid, did, mark = int(qid), int(did), int(mark)
                if mark==0:
                  pass
                  #continue
            else:
                parts = line.split(",")
                if len(parts)!=2:
                    continue
                qid, did = parts
                mark=-1
                qid, did, mark = int(qid), int(did), int(mark)
            flag, features = feature_func(qid, did, feature_type=1) #uses global vars
            features = np.array(features)
            if flag:
                X_train[i, :] = features
                y_train[i] = mark
                qids[i] = qid
                trainDidToLine[(qid,did)] = i
                
                qidToDocs[qid].append(did)
                i+=1
            else:
                rejectedDids[qid].append(did)
        X_train = X_train[0:i, :]
        y_train = y_train[0:i]
        qids = qids[0:i]
    return X_train, y_train, qids, trainDidToLine, qidToDocs, rejectedDids


In [ ]:
import gc
gc.collect()
X_train, y_train, q_train, trainDidToLine, trainQidToDocs, rejectedTrainDids = makeTrain("train.marks.tsv", N_FEATURES, 
                                                                      isTest=False, 
                                                                      feature_func=getFeaturesByIds)
X_test, _, q_test, testDidToLine, testQidToDocs, rejectedTestDids = makeTrain("sample.csv", N_FEATURES, 
                                                            isTest=True, 
                                                            feature_func=getFeaturesByIds)
print(X_train.shape, y_train.shape, q_train.shape, len(trainDidToLine))
print(X_test.shape, 0, q_test.shape, len(testDidToLine))

(198785, 290) (198785,) (198785,) 198785
(397120, 290) 0 (397120,) 397120


In [ ]:
#del alldomains
#del foundQueries
#del foundQueriesDomains
#del tfidf_features
#del tfidf_ngram_features
#del tf_content

#del alldocs


def removeZeros(X_train, X_test):
    idx = np.argwhere(np.all(X_train[..., :] == 0, axis=0))
    return np.delete(X_train, idx, axis=1),np.delete(X_test, idx, axis=1)
    
def removeBad(X_train, X_test, fi):
    idx = np.argwhere(fi<0)
    print(len(idx))
    return np.delete(X_train, idx, axis=1),np.delete(X_test, idx, axis=1)

In [ ]:
#del bm25_features


gc.collect()

254

In [ ]:
gc.collect()

0

In [ ]:
X_train, X_test = removeZeros(X_train, X_test)

41


In [ ]:
print(X_train.shape, y_train.shape, q_train.shape, len(trainDidToLine))
print(X_test.shape, 0, q_test.shape, len(testDidToLine))

(198785, 169) (198785,) (198785,) 198785
(397120, 169) 0 (397120,) 397120


In [ ]:

from catboost import CatBoostClassifier,cv, CatBoost
import catboost
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

def split_train_val(X, y, qids, fr):
    ind = int(y.shape[0] * fr)
    X_train = X[0:ind, :]
    X_test = X[ind:, :]
    y_train = y[0:ind]
    y_test = y[ind:]
    qids_train = qids[0:ind]
    qids_test = qids[ind:]
    return X_train, X_test, y_train, y_test, qids_train, qids_test



X_trainp, X_validation, y_trainp, y_validation, q_trainp, q_valp  = split_train_val(X_train, y_train, q_train, 0.75)


In [ ]:
def qidsToGroupSizes(qids):
    group_sizes=[]
    cur_qid = qids[0]
    size=0
    for qid in qids:
        if qid==cur_qid:
            size+=1
        else:
            group_sizes.append(size)
            cur_qid = qid
            size=1
    if size!=0:
        group_sizes.append(size)
    return group_sizes
train_group_sizes=np.array(qidsToGroupSizes(q_trainp))
train_all_group_sizes=np.array(qidsToGroupSizes(q_train))
val_group_sizes  = np.array(qidsToGroupSizes(q_valp))
test_group_sizes = np.array(qidsToGroupSizes(q_test))
len(q_trainp), np.sum(train_group_sizes), len(train_group_sizes)

(149088, 149088, 1591)

In [ ]:
train_pool = catboost.Pool(data=X_trainp, label=y_trainp , group_id = q_trainp)
train_pool_all = catboost.Pool(data=X_train, label=y_train , group_id = q_train) #final rank
train_pool=train_pool_all
validation_pool = catboost.Pool(data=X_validation, label=y_validation , group_id = q_valp)
test_pool = catboost.Pool(data=X_test, group_id = q_test)
param = {'loss_function':'YetiRank',
         #'learning_rate': .00001,
         'learning_rate': 0.3,
         'iterations': 1500,
         'depth': 5,
         'use_best_model':True,
        'task_type' : "CPU",
        'custom_metric': "NDCG:top=5",
         #'eval_metric': "NDCG:top=5"
         #'one_hot_max_size' : 20,
         #'bagging_temperature' : 0.75,
         #'max_ctr_complexity' : 4 
        }
model = CatBoost(param)
model.fit(
    train_pool,
    eval_set=validation_pool,
    #logging_level='Silent',
    plot=True
);


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.9217240	best: 0.9217240 (0)	total: 525ms	remaining: 13m 7s
1:	test: 0.9328771	best: 0.9328771 (1)	total: 1.03s	remaining: 12m 50s
2:	test: 0.9380970	best: 0.9380970 (2)	total: 1.54s	remaining: 12m 49s
3:	test: 0.9444074	best: 0.9444074 (3)	total: 2.04s	remaining: 12m 41s
4:	test: 0.9457759	best: 0.9457759 (4)	total: 2.55s	remaining: 12m 42s
5:	test: 0.9478644	best: 0.9478644 (5)	total: 3.05s	remaining: 12m 40s
6:	test: 0.9483919	best: 0.9483919 (6)	total: 3.57s	remaining: 12m 42s
7:	test: 0.9493171	best: 0.9493171 (7)	total: 4.1s	remaining: 12m 43s
8:	test: 0.9515016	best: 0.9515016 (8)	total: 4.61s	remaining: 12m 43s
9:	test: 0.9525438	best: 0.9525438 (9)	total: 5.14s	remaining: 12m 45s
10:	test: 0.9528509	best: 0.9528509 (10)	total: 5.66s	remaining: 12m 46s
11:	test: 0.9531596	best: 0.9531596 (11)	total: 6.18s	remaining: 12m 46s
12:	test: 0.9533103	best: 0.9533103 (12)	total: 6.7s	remaining: 12m 46s
13:	test: 0.9545882	best: 0.9545882 (13)	total: 7.23s	remaining: 12m 47s
1

In [ ]:
y=model.predict(validation_pool)


In [ ]:
fi = model.get_feature_importance(data=validation_pool,
                       type="FeatureImportance",
                       prettified=False,
                       thread_count=-1,
                       verbose=False)

In [ ]:
X_train, X_test = removeZeros(X_train, X_test)
X_train, X_test = removeBad(X_train, X_test, fi)


In [ ]:
X_trainp, X_validation, y_trainp, y_validation, q_trainp, q_valp  = split_train_val(X_train, y_train, q_train, 0.75)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#plt.bar(np.arange(len(fi)), fi )

In [ ]:
import lightgbm as lgb
gbm = []
gc.collect()
gbm = lgb.LGBMRanker(n_estimators=500,  objective="lambdarank", learning_rate=0.05, max_depth=60)
#gbm = gbm.fit(X_trainp, y_trainp, group=train_group_sizes, eval_metric="ndcg@5",eval_at=[5],
#        eval_set=[(X_validation, y_validation)], eval_group=[val_group_sizes], 
#        )
gbm.fit(X_train, y_train, group=train_all_group_sizes, eval_metric="ndcg@5",eval_at=[5],
        eval_set=[(X_validation, y_validation)], eval_group=[val_group_sizes], 
        )


#import matplotlib.pyplot as plt 
#ax = lgb.plot_importance(gbm,height=0.1, max_num_features=20)

#plt.show()

In [ ]:
#y_test = model.predict(test_pool)
y_test=[]
y_test = gbm.predict(X_test, group=test_group_sizes)
error_cnt=0
with open("res16_132_all_gbm_ft_gbm_npw_ctb_idf_new_engine_2.txt", "w") as g:
    g.write("QueryId,DocumentId\n")
    for qid in range(6310+1):
        qpred = []
        for did in testQidToDocs[qid]:
            if (qid,did) in testDidToLine:
                qpred.append( (did, float(y_test[testDidToLine[ (qid,did) ]]))) 
            else:
                qpred.append((did,0))
                error_cnt+=1
        for did in rejectedTestDids[qid]:
            if (qid,did) in bm25_features:
                bm25_score = bm25_features[(qid,did)][0]
                qpred.append( (did, -100000 + bm25_score)) 
                error_cnt+=1
        sorted_qpred = sorted(qpred, key=operator.itemgetter(1), reverse=True)
        for did,rat in sorted_qpred:
            g.write(str(qid) + "," + str(did) + "\n") #+ "," + str(rat) + 
print(error_cnt)